In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round



In [2]:
#  Set Hadoop home directory (update the path as needed)
os.environ["HADOOP_HOME"] = "C:/hadoop/hadoop-3.2.2"
os.environ["HADOOP_COMMON_LIB_NATIVE_DIR"] = os.path.join(os.environ["HADOOP_HOME"], "lib/native")
os.environ["PATH"] = os.environ["HADOOP_HOME"] + "/bin:" + os.environ["PATH"]

In [7]:
# Stop any existing Spark contexts
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except Exception as e:
    print(e)

# Initialize a new Spark session
spark = SparkSession.builder.appName("HomeSales").getOrCreate()

name 'SparkContext' is not defined


KeyboardInterrupt: 

In [6]:
# Read the data 
df = spark.read.csv("C:/Users/n.rennie/Documents/Module 22 Challenge/Home_Sales/home_sales_revised.csv", header=True, inferSchema=True)

NameError: name 'spark' is not defined

In [1]:
# Show the schema to check column names
df.printSchema()

NameError: name 'df' is not defined

In [ ]:
# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


In [ ]:
# What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query1 = """
SELECT year(date) as year, round(avg(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date)
"""
result1 = spark.sql(query1)
result1.show()



+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [ ]:
#  What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query2 = """
SELECT year(date) as year, round(avg(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date)
"""
result2 = spark.sql(query2)
result2.show()



+----+---------+
|year|avg_price|
+----+---------+
|2022|292725.69|
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
+----+---------+



In [ ]:
# Average price for a home with specific features each year
query3 = """
SELECT year(date) as year, round(avg(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year(date)
"""
result3 = spark.sql(query3)
result3.show()

+----+---------+
|year|avg_price|
+----+---------+
|2022|290242.99|
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
+----+---------+



In [ ]:
# Average price of a home for each year the home was built
query4 = """
SELECT date_built, round(avg(price), 2) as avg_price
FROM home_sales
GROUP BY date_built
ORDER BY date_built
"""
result4 = spark.sql(query4)
result4.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|309751.17|
|      2011|314804.32|
|      2012|314072.77|
|      2013| 317597.6|
|      2014|316395.73|
|      2015|315322.64|
|      2016|315471.57|
|      2017|315799.06|
+----------+---------+



In [ ]:
#  What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

# Average price per "view" rating with average price ≥ $350,000
query5 = """
SELECT view, round(avg(price), 2) as avg_price
FROM home_sales
GROUP BY view
HAVING avg(price) >= 350000
"""
start_time = time.time()
result5 = spark.sql(query5)
result5.show()
print("Runtime: %s seconds" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

Runtime: 0.28717923164367676 seconds


In [ ]:
#  Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [ ]:
#  Check if the table is cached.
print("Is home_sales cached? ", spark.catalog.isCached("home_sales"))

Is home_sales cached?  True


In [ ]:
#  Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time_cached = time.time()
result5_cached = spark.sql(query5)
result5_cached.show()
print("Cached Runtime: %s seconds" % (time.time() - start_time_cached))


+----+----------+
|view| avg_price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

Cached Runtime: 0.30167722702026367 seconds


In [ ]:
# Write partitioned parquet data, overwrite if the path already exists
df.write.mode("overwrite").partitionBy("date_built").parquet("home_sales_parquet")


Py4JJavaError: An error occurred while calling o209.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [ ]:
#  Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("home_sales_parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/c:/Users/n.rennie/Documents/Module 22 Challenge/Starter_Code/home_sales_parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
#  Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_parquet")
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
#  Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
import time

# Start measuring the time
start_time = time.time()

# Define and run the query
query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
result = spark.sql(query)
result.show()

# Stop measuring the time
end_time = time.time()

# Print the runtime
print("--- %s seconds ---" % (end_time - start_time))

In [ ]:
#  Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

In [ ]:
# Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is home_sales table cached? {is_cached}")
